# RNN - Pure Numpy

In [8]:
import numpy as np
data = open('Dataset.txt', 'r').read()
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)

print "data has %d chars and %d unique"%(data_size, vocab_size)

data has 137629 chars and 81 unique


We add the data into our code in the first cell as shown above. Clearly as is evident we have 137629 characters in our data file and 81 of them are unique. What are those characters? Well, let us find out :-

In [9]:
char_to_ix = { ch:i for i,ch in enumerate(chars)}
ix_to_char = { i:ch for i, ch in enumerate(chars)}
print char_to_ix
print ix_to_char

{'\n': 0, 'C': 31, '!': 3, ' ': 4, '"': 5, '%': 6, '$': 7, "'": 8, ')': 9, '(': 10, '*': 11, '-': 12, ',': 13, '/': 2, '.': 15, '1': 16, '0': 17, '3': 18, '2': 19, '5': 20, '4': 21, '7': 22, '6': 23, '9': 24, '8': 25, ';': 26, ':': 27, '?': 28, 'A': 29, '@': 30, '\xc3': 1, 'B': 32, 'E': 33, 'D': 34, 'G': 35, 'F': 36, 'I': 37, 'H': 38, 'K': 39, 'J': 40, 'M': 41, 'L': 42, 'O': 43, 'N': 44, 'Q': 45, 'P': 46, 'S': 47, 'R': 48, 'U': 49, 'T': 50, 'W': 51, 'V': 52, 'Y': 53, 'X': 54, 'd': 59, 'a': 55, 'c': 56, 'b': 57, 'e': 58, '\xa7': 14, 'g': 60, 'f': 61, 'i': 62, 'h': 63, 'k': 64, 'j': 65, 'm': 66, 'l': 67, 'o': 68, 'n': 69, 'q': 70, 'p': 71, 's': 72, 'r': 73, 'u': 74, 't': 75, 'w': 76, 'v': 77, 'y': 78, 'x': 79, 'z': 80}
{0: '\n', 1: '\xc3', 2: '/', 3: '!', 4: ' ', 5: '"', 6: '%', 7: '$', 8: "'", 9: ')', 10: '(', 11: '*', 12: '-', 13: ',', 14: '\xa7', 15: '.', 16: '1', 17: '0', 18: '3', 19: '2', 20: '5', 21: '4', 22: '7', 23: '6', 24: '9', 25: '8', 26: ';', 27: ':', 28: '?', 29: 'A', 30: '

In [16]:
def vector_for_a_char(char):
    vector = np.zeros((vocab_size, 1))
    vector[char_to_ix[char]] = 1
    return vector

def char_for_a_vector(vector):
    ix = np.argmax(vector)
    return ix_to_char[ix]

In [17]:
hidden_size = 100
seq_length = 25
learning_rate = 1e-1

Wxh = np.random.randn(hidden_size, vocab_size) * 0.01 #input to hidden
Whh = np.random.randn(hidden_size, hidden_size) * 0.01 #input to hidden
Why = np.random.randn(vocab_size, hidden_size) * 0.01 #input to hidden
bh = np.zeros((hidden_size, 1))
by = np.zeros((vocab_size, 1))

In [35]:
def loss_fun(inputs, targets, hprev):
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = hprev
    loss = 0
    
    #Performing the forward pass
    for t in xrange(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1))
        xs[t][inputs[t]] = 1
        
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh)
        ys[t] = np.dot(Why, hs[t]) + by
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t]))
        
        loss += -np.log(ps[t][targets[t], 0])
        
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    
    for t in reversed(xrange(len(inputs))):
        
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1
        
        dWhy += np.dot(dy, hs[t].T)
        dby  += dy
        
        dh = np.dot(Why.T, dy) + dhnext
        dhraw = (1 - hs[t]*hs[t]) * dh
        
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        
        dhnext = np.dot(Whh.T, dhraw)
        
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) 
    
    
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs) - 1]     

In [36]:
def sample(h, seed_ix, n):
    
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    
    ixes = []
    
    
    for t in xrange(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        
        
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        
        ixes.append(ix)
        
    
    text = ''.join(ix_to_char[ix] for ix in ixes)
    print '----\n %s \n----' % (text, )

hprev = np.zeros((hidden_size, 1))
sample(hprev, char_to_ix['a'], 200)

----
 IyQGT.@)tF!p5S$Bu(0$P2 bYMt2y36SPf'%/oD@u4enC/zc5pEs*uMg-;MVk�o/:WPx;oFy"2S1�@I6Ru1$6z(cBSmc;EyCm%UTR2-(A,$Gdn5$?s@U656��0*v(@lmhMm/ibEFW*rBH.!QxIbhY7@!rFQbEA2(Y4.u2"7W"qlAiTQlj?stgcCPi2fJ5"Ae;/Dy9$L5 
----


In [37]:
p=0  
inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
print "inputs", inputs
targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
print "targets", targets

inputs [43, 69, 58, 4, 66, 68, 73, 69, 62, 69, 60, 13, 4, 76, 63, 58, 69, 4, 35, 73, 58, 60, 68, 73, 4]
targets [69, 58, 4, 66, 68, 73, 69, 62, 69, 60, 13, 4, 76, 63, 58, 69, 4, 35, 73, 58, 60, 68, 73, 4, 47]


In [38]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by)

smooth_loss = -np.log(1.0/vocab_size)*seq_length

In [39]:
while n<=1000*100:
    if p+seq_length+1 >= len(data) or n == 0:
        hprev = np.zeros((hidden_size,1))
        p = 0
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
    
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = loss_fun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    
    if n % 1000 == 0:
        print 'iter %d, loss: %f' % (n, smooth_loss) # print progress
        sample(hprev, inputs[0], 200)
        
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by],
                                [dWxh, dWhh, dWhy, dbh, dby],
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8)
        
    p += seq_length
    n += 1

    
    

iter 0, loss: 109.861235
----
 f?@@w xWr((7IRS)9@/vWkA:9ji
gSU')1eFh@rL.?D$x)�mDECpe0AxiGb)AH4Fi(l;t7hH)Sn"64S6zI1k'2v2t9
-R�)6erOalx1'S$1N*Ybc25y�(I4iL9R"kiu�FiFzM0
h$%.h�31*RDUGtbj .C9P)Q� X.hsO$id)z(6-nIasJN�lRO1l
.7 y)05@"Ce7O3 
----
iter 1000, loss: 85.838047
----
 h so thetw shi whe whem tiof f le norr l d yhims ulod fese tpe: py pade ame,ve whoug t thed enunog pas vofet nt ran, hncurolider  Wge oflg hveo ehiey fihf beullve lkrf bom toas,ne t lo t beme hit;afs  
----
iter 2000, loss: 69.472936
----
 oi he hr oreca sideghand torarg wols mafdeous hiver sowhongesd.l hork soup osae soins meadllt ike uatts whift thang ssersilm he, s fauthally hiter auk ehly halyooweth uaxjelgouckeme.s then ghus vok we 
----
iter 3000, loss: 61.034718
----
 nhey, shell ind hant ar oubiwy whrowsrseert in te s cos reind, He l's;thswabeins; to asems sood tuthery wisrey ndy thure; toort atcaly poudsout ay oomelk lougcs abiry locig aris; omlkiwy toout toose t 
----
iter 4000, loss: 56.945008
----
  lastoin 